![EM German Logo](https://github.com/jphme/EM_German/blob/main/em_model_logo_web.jpeg?raw=true)
# EM German Mistral GGUF Demo
Please visit the [Github Repository](https://github.com/jphme/EM_German) for all model resources and if you have questions or feedback!

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [ ]:
!pip install huggingface_hub

In [3]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download

In [4]:
modelpath=hf_hub_download(repo_id='TheBloke/em_german_mistral_v01-GGUF', filename="em_german_mistral_v01.Q8_0.gguf")

In [5]:
llm = Llama(
      model_path=modelpath,
      n_ctx=2048,
      n_batch=1024,
      n_gpu_layers=-1,
      #verbose=False
      )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [13]:
instruction="Nenne 5 Unternehmen aus dem DAX!"
output=llm(
    f"Du bist ein hilfreicher Assistent. USER: {instruction} ASSISTANT:",
    max_tokens=500,
    temperature=0.6)
print(output['choices'][0]['text'])

Llama.generate: prefix-match hit


 Der DAX ist eine Liste der 30 größten und liquidesten Unternehmen an der Frankfurter Börse. Hier sind fünf davon:

1. Deutsche Telekom AG - Ein international tätiger deutscher Mobilfunk- und Telekommunikationsanbieter mit Hauptsitz in Bonn.

2. SAP SE - Weltweit führender Anbieter von Geschäftssoftware, der seinen Sitz in Walldorf hat.

3. BASF SE - Ein global tätiger deutscher Chemie- und Produktionskonzern mit Hauptsitz in Ludwigshafen am Rhein.

4. Bayer AG - Eine weltweit führende Life-Science-Unternehmensgruppe, die ihren Sitz in Leverkusen hat.

5. Volkswagen AG - Ein automobilherstellender Konzern mit Hauptsitz in Wolfsburg und einer globalen Präsenz.


# Grammar - Structured Output Demo

In [6]:
import json
from llama_cpp import LlamaGrammar

In [7]:
schema={
  'type': 'object',
  'properties': {
    'name': {'type': 'string'},
    'ort': {'type': 'string'},
    'jahr': {'type': 'integer'}},
  'required': ['name', 'ort', 'jahr'],
}

with open('json_schema.txt','w') as file:
    json.dump(schema, file)

In [ ]:
!wget https://raw.githubusercontent.com/ggerganov/llama.cpp/master/examples/json-schema-to-grammar.py

In [9]:
!python json-schema-to-grammar.py json_schema.txt > json_grammar.gbnf

In [10]:
grammar=LlamaGrammar.from_file('json_grammar.gbnf')

space ::= space_1 
space_1 ::= [ ] | 
integer ::= integer_3 space 
integer_3 ::= integer_4 integer_5 
integer_4 ::= [-] | 
integer_5 ::= [0-9] | [1-9] integer_6 
integer_6 ::= [0-9] integer_6 | 
string ::= ["] string_10 ["] space 
string_8 ::= [^"\] | [\] string_9 
string_9 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_10 ::= string_8 string_10 | 
root ::= [{] space ["] [j] [a] [h] [r] ["] space [:] space integer [,] space ["] [n] [a] [m] [e] ["] space [:] space string [,] space ["] [o] [r] [t] ["] space [:] space string [}] space 


from_string grammar:



In [11]:
instruction="Max ging im Jahr, als die Corona-Krise startete, in die Stadt mit dem Eiffelturm."
output=llm(
    f"Bitte gebe gültiges JSON aus. USER: {instruction} ASSISTANT:",
    max_tokens=200,
    temperature=0.3,
    grammar=grammar)
json.loads(output['choices'][0]['text'])

{'jahr': 2020, 'name': 'Max', 'ort': 'Paris'}